# Classificazione con alberi di decisione

Nel seguente notebook verrà creato un albero di decisione per eseguire una classificazione tra pazienti diabetici e non.

## Dataset
Il dataset utilizzato è Pima Indians Diabetes Database, contiene dati relativi a 768 pazienti, per ognuno dei quali sono presenti 8 diversi attributi:
- Pregnancies	
- Glucose	
- BloodPressure	
- SkinThickness	
- Insulin	
- BMI	
- DiabetesPedigreeFunction
- Age
- Outcome: la classificazione, 1 per i pazienti diabetici 0 altrimenti


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

dataset = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
dataset.head()

In [ ]:
# divido i dati relativi agli features (attributi) dalle classi (Outcome)
feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age','Glucose','BloodPressure','DiabetesPedigreeFunction']
x = dataset[feature_cols] # features
y = dataset.Outcome # classi
# divido il dataset in training set e test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [ ]:
# creo il classificatore DecisionTreeClassifier
dtc = DecisionTreeClassifier()
# training del classificatore
dtc = dtc.fit(x_train,y_train)
# eseguo la predizione dei dati del test set
y_pred = dtc.predict(x_test)
# calcolo l'accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

## Visualizzazione dell'albero ottenuto
Il seguente codice si occupa di mostrare l'albero di decisione creato. La funzione export_graphviz si occupa di traformare il classificatore (dtc) in un file dot che succesivamente viene convertito in un immagine grazie al package pydotplus.

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image 
!pip install pydotplus
import pydotplus

dot_data = StringIO()
export_graphviz(dtc, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('diabetes.png')
Image(graph.create_png())

Possiamo notare come l'albero ottenuto sia molto esteso e di difficile comprensione, ora procederemo ad ottimizzare le performance del classificatore.

## Ottimizzazione
Possiamo incrementare le performance del classificatore cambiando alcuni parametri, analizzando la [documentazione](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), troviamo i seguneti parametri per il classificatore:

- criterion : consente di utilizzare un criterio differente per la selezione degli attributi. Sono disponibili i valori “gini” per scegliere gli attributi in base al Gini index e “entropy” per scegliere gli attributi in base all'information gain.

- splitter : consente di scegliere la strategia di divisione (split strategy). I valori disponibili sono “best” per scegliere una strategia ottima o “random” per scegliere una strategia random.

- max_depth : Questo paramtro permette di impostare un limite alla profondità massima dell'albero. Se viene settato a None, allora i nodi vengono espansi fino a trovare foglie pure. Utilizzando valori alti, quindi sviluppando l'abero in profondità rischiamo l'overfitting, mentre con valori troppo bassi rischiamo di ottenere un classificatore sottospecializzato (underfitting).

In [ ]:
# creo il classificatore DecisionTreeClassifier
dtc_optimized = DecisionTreeClassifier(criterion="entropy", max_depth=3)
# training del classificatore
dtc_optimized = dtc_optimized.fit(x_train,y_train)
# eseguo la predizione dei dati del test set
y_pred = dtc_optimized.predict(x_test)
# calcolo l'accuracy per il classificatore ottimizzato
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:

dot_data = StringIO()
export_graphviz(dtc_optimized, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('diabetes.png')
Image(graph.create_png())

Possiamo notare come sia migliorata la situazione, potando l'albero abbiamo incrementato l'accuracy circa del 10%. 